In [ ]:
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import LabelEncoder


drive.mount('/content/drive')




Mounted at /content/drive


## **Question 1**

## **Demonstrating the data preparation & feature extraction process**

In [ ]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_21.csv')

In [ ]:
df2=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_22.csv')

<ipython-input-43-23c7cb22266c>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_22.csv')


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns.tolist()

## **Removing useless varaiables**

In [ ]:
df.drop(columns=['player_url','player_face_url','club_logo_url','club_flag_url','nation_flag_url'], inplace = True)

In [ ]:
threshold = 0.3 * len(df)  # 30% threshold
df_no_useless_variables = df.dropna(axis=1, thresh=threshold)# dropping columns with more than 30% of their values missing

In [ ]:
df_no_useless_variables.columns.tolist()

In [ ]:
df_no_useless_variables.shape

(18944, 98)

In [ ]:
df = df_no_useless_variables

## **Imputation for numerical variables using KNN Imputer**

In [ ]:
df.dtypes.tolist() # trying to see the different datatypes in the dataframe

In [ ]:
from pandas.core.arrays import categorical
numericVariables = df.select_dtypes(include=['int64','float64'])
categoricalVariables = df.select_dtypes(include=['object']) # separating the dataframe into numeric and categorical variables

In [ ]:
from sklearn.impute import KNNImputer


In [ ]:
knn_imputer = KNNImputer(n_neighbors=5)
numericVariables_imputed = knn_imputer.fit_transform(numericVariables)

In [ ]:
numericVariables_imputed_df = pd.DataFrame(numericVariables_imputed,columns=numericVariables.columns)

In [ ]:
numericVariables_imputed_df

In [ ]:
numericVariables_imputed_df.isnull().sum().tolist() # checking to see if there are any null calues still

# **Imputation for categorical variables using Forward Filling**

In [ ]:
categoricalVariables_imputed = categoricalVariables.ffill()

In [ ]:
categoricalVariables_imputed_df = pd.DataFrame(categoricalVariables_imputed, columns =categoricalVariables.columns)

In [ ]:
categoricalVariables_imputed_df

In [ ]:
categoricalVariables_imputed_df.isnull().sum().tolist() # checking to see if there are any null calues still

## **Encoding the categorical variables using Onehot encoding**

In [ ]:
#label_encoder = LabelEncoder()

In [ ]:
#categoricalVariables_df_encoded = categoricalVariables_imputed_df.apply(label_encoder.fit_transform)

In [ ]:
categoricalVariables_df_encoded = pd.get_dummies(categoricalVariables_imputed_df, prefix='categorical')

In [ ]:
categoricalVariables_df_encoded

## **Creating a dataframe that contains both the numeric and categorical variables**

In [ ]:
new_df = pd.concat([numericVariables_imputed_df, categoricalVariables_df_encoded], axis=1)

In [ ]:
new_df

## **Separating the independent and dependent variables**

In [ ]:
y = new_df['overall']
X = new_df.drop('overall', axis = 1)

## **Question 2**

# **Creating feature subsets which show better correlation with the overall rating and scaled the independent variables.**

**Created feature subsets which show better correlation with the overall rating**

In [ ]:
X_df =  pd.DataFrame(X)

In [ ]:
correlations = X_df.corrwith(y)

In [ ]:
correlations

In [ ]:
selected_features = correlations[(correlations >= 0.4) | (correlations <= -0.4)] # selecting features with correlations greater than or
                                                                                 # equal to 0.4 and less than or equal to -0.4


In [ ]:
selected_features.sort_values(ascending=False) # sorting the selected features in ascending order

In [ ]:
feature_subset = X_df[selected_features.index]
X = feature_subset # putting the selected features in X

In [ ]:
X_df

In [ ]:
X.info()

## **Scaling the independent variables**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
scaled=sc.fit_transform(X)

In [ ]:
scaled.tolist()

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42)
Xtrain.shape

(15155, 24)

## **Question 3**

# **Cross Validation for the Random Forest Regressor**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

cv = KFold(n_splits=3)

RFM_PARAMETERS = {"max_depth":[2,5, 6, None],
              "max_features":[1.0,0.3,15],
              "min_samples_split": [2, 5, 10],
              "n_estimators":[100,500,300]}

full = RandomForestRegressor(n_jobs= -1)
RFM_model_gs = GridSearchCV (full, param_grid= RFM_PARAMETERS, cv=cv, scoring= "accuracy")
RFM_model_gs.fit ( Xtrain, Ytrain)
RFMy_pred= RFM_model_gs.predict(Xtest)

In [75]:
RFMy_pred

array([64.05880348, 64.05880348, 57.01865452, ..., 70.37413692,
       63.37049028, 64.05880348])

## **Cross Validation for the XGBoost Regressor**

In [73]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
cv = KFold(n_splits=3)
PARAMETERS = {"subsample":[0.5, 0.75, 1],
              "max_depth":[2,5, 6, 12],
              "learning_rate":[0.3, 0.1, 0.03],
              "n_estimators":[100,500,1000]}

XGBModel = xgb.XGBRegressor(n_jobs= -1)

XGBmodel_gs = GridSearchCV(XGBModel, param_grid=PARAMETERS, cv=cv, scoring='neg_mean_squared_error')
XGBmodel_gs.fit(Xtrain,Ytrain)
XGBy_pred= XGBmodel_gs.predict(Xtest)

## **Cross Validation for the Gradient Boosting Regressor**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np


model = GradientBoostingRegressor()

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]}

GBgrid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
GBgrid_search.fit(Xtrain, Ytrain)
GBy_pred = GBgrid_search.predict(Xtest)


## **Question 4**

In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculating the MAE of the RandomForestRegressor

In [ ]:
mae_RFM = mean_absolute_error(Ytest, RFMy_pred )
print(f"Mean Absolute Error (MAE): {mae_RFM}")


Mean Absolute Error (MAE): 2.506425477795331


# Calculating the MAE of the XGBoostRegressor

In [74]:
mae_XGB= mean_absolute_error(Ytest, XGBy_pred )
print(f"Mean Absolute Error (MAE): {mae_XGB}")


Mean Absolute Error (MAE): 0.27057743651409455


# Calculating the MAE of the GradientBoostingRegressor

In [ ]:
mae_GB= mean_absolute_error(Ytest, GBy_pred )
print(f"Mean Absolute Error (MAE): {mae_GB}")



Mean Absolute Error (MAE): 0.356579606589903


## **Question 5**

In [ ]:
from pandas.core.arrays import categorical
numeric_TestVariables = df.select_dtypes(include=['int64','float64'])
categorical_TestVariables = df.select_dtypes(include=['object'])

## **Numeric** **Imputation for the 2022 dataset variables using KNN Imputation**

In [ ]:
numeric_TestVariables_imputed = knn_imputer.fit_transform(numeric_TestVariables)

In [ ]:
numeric_TestVariables_imputed_df = pd.DataFrame(numeric_TestVariables_imputed,columns=numeric_TestVariables.columns)

## **Imputation for the Categorical Variables of the 2022 dataset**

In [ ]:
categorical_TestVariables_imputed = categorical_TestVariables.ffill()

In [ ]:
categorical_TestVariables_imputed_df = pd.DataFrame(categorical_TestVariables_imputed, columns =categorical_TestVariables.columns)

## **Encoding the Categorical variables of the 2022 dataset**

In [ ]:
categorical_TestVariables_df_encoded = pd.get_dummies(categorical_TestVariables_imputed_df, prefix='categorical')

In [ ]:
new_df2 = pd.concat([numeric_TestVariables_imputed_df, categorical_TestVariables_df_encoded], axis=1)

## **Separating the independent and dependent variables in the 2022 dataset**

In [ ]:
y2 = new_df2['overall']

In [ ]:
new_df2

In [ ]:
testData =new_df2[selected_features.index] # Extracting features with high correlation from the 2022 dataset based on observations in the 2021 dataset.
#Extracting the corresponding features from the 2022 dataset in the 2021 dataset and putting them in a dataframe


In [ ]:
testData.info()

## **Scaling the independent variables of the 2022 dataset**

In [ ]:
scaled2=sc.fit_transform(testData)

## **Testing the choosen model using the choosen columns from the 2022 dataset**

In [ ]:
XGBy_pred2= XGBmodel_gs.predict(testData)

## **Calulating the Mean Absolute Error**

In [77]:
mae_XGB= mean_absolute_error(y2, XGBy_pred2 )
print(f"Mean Absolute Error (MAE): {mae_XGB}")


Mean Absolute Error (MAE): 0.05959793804465113


# **Saving the model in a pickle file**

In [78]:
import pickle
XGBmodel_gs_model = XGBmodel_gs

# Specify the filename for the pickle file
pickle_filename = 'XGBmodel_gs_model.pkl'

# Save the model to a pickle file
with open(pickle_filename, 'wb') as file:
    pickle.dump(XGBmodel_gs_model, file)